In [9]:
"""Code assumes the ability to train using a GPU with CUDA.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from advertorch.attacks import GradientSignAttack, CarliniWagnerL2Attack, PGDAttack
import matplotlib.pyplot as plt
import utils.adv_ex_utils as aus
import utils.interp_generators as igs
import utils.utils as utils
from utils.models import LeNet, DDNet
from utils.data_loaders import DataLoader

# makes default tensor a CUDA tensor so GPU can be used
device = torch.device(1 if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
if device != 'cpu':
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

### Define train and test functions 

In [15]:
def train():
    net.train()
    
    for batch_idx, (samples, labels) in enumerate(train_loader):
        # sends to GPU, i.e. essentially converts from torch.FloatTensor to torch.cuda.FloatTensor
        samples, labels = samples.to(device), labels.to(device)
        
        # expand dataset with adversarial examples if adversary specified
        if adversary != None:
            adv_samples, adv_labels = aus.generate_adv_exs(samples, labels, adversary)
            samples, labels = torch.cat([samples, adv_samples], 0), torch.cat([labels, adv_labels], 0)
                
        optimizer.zero_grad()
        
        output = net(samples)
        
        loss = utils.my_loss(output, labels, net=net, optimizer=optimizer,
                             alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, 
                             bp_mat=tr, alpha_ir1=alpha_ir1, alpha_ir2=alpha_ir2, scale=scale)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            j = utils.avg_norm_jacobian(net, samples, output.shape[1], tr, for_loss=False)
            i,_ = utils.norm_diff_interp(net, samples, labels)
            print(f'\tLoss: {loss.item():.6f}, Average norm of Jacobian: {j:6f}, Norm of difference in interpretations: {i:6f}')

In [16]:
def test():
    net.eval()
    test_loss = 0
    correct = 0
    
    for samples, labels in test_loader:
        samples, labels = samples.to(device), labels.to(device)
        output = net(samples)
        test_loss += utils.my_loss(output, labels, net=net, optimizer=optimizer,
                             alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, 
                             bp_mat=tr, alpha_ir1=alpha_ir1, alpha_ir2=alpha_ir2, scale=scale).item()
        # output is a tensor, .data retrieves its data, max returns the index of the highest valued element
        preds = output.data.max(1, keepdim=True)[1]
        correct += preds.eq(labels.data.view_as(preds)).sum().item()
                
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * float(correct / len(test_loader.dataset))
    
    print(f'\tTest set accuracy: ({test_accuracy:.2f}%)')

### Training with interpretation regularization

In [17]:
dataset = 'CIFAR-10'

dl = DataLoader(dataset=dataset)
train_loader = dl.train_loader
test_loader = dl.test_loader
tr_batch_size = dl.tr_batch_size
te_batch_size = dl.te_batch_size

Files already downloaded and verified
Files already downloaded and verified


In [18]:
# training details
n_epochs = 30
log_interval = 200
training_round = 1
torch.manual_seed(7)
adversary = None

In [20]:
# loss hyperparameters
alpha_wd = 0
alpha_jr = 0

alpha_ir1s = [.0000001, .000001, .00001]
alpha_ir2s = [.00000001, .0000001, .000001]
scale = .1

for alpha_ir1 in alpha_ir1s:
    for alpha_ir2 in alpha_ir2s:
        model_name = f'ir1{alpha_ir1}_ir2{alpha_ir2}_scale{scale}'
        # instantiate model and optimizer
        learning_rate = 0.01
        momentum = 0.9
        if dataset == 'MNIST':
            net = LeNet()
        elif dataset == 'CIFAR-10':
            net = DDNet()
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
        lr_decayer = StepLR(optimizer, step_size=10, gamma=0.1)

        # make model CUDA enabled and define GPU/device to use
        net.cuda()

        tr = utils.bp_matrix(tr_batch_size, 10)
        te = utils.bp_matrix(te_batch_size, 10)
        
        print(f'Beginning training for {model_name}:')
        for epoch in range(1, n_epochs + 1):
            print(f'Epoch #{epoch}')
            train()
            test()
            lr_decayer.step()

        torch.save(net.state_dict(), f'trained_models/{dataset}/training_round_1/{model_name}')

Beginning training for ir11e-07_ir21e-08_scale0.1:
Epoch #1
	Loss: 2.310898, Average norm of Jacobian: 0.001357, Norm of difference in interpretations: 0.028811
	Loss: 1.851208, Average norm of Jacobian: 0.500148, Norm of difference in interpretations: 0.366150
	Loss: 1.626328, Average norm of Jacobian: 0.869465, Norm of difference in interpretations: 0.530731
	Loss: 1.533733, Average norm of Jacobian: 2.167679, Norm of difference in interpretations: 0.789481
	Test set accuracy: (46.87%)
Epoch #2
	Loss: 1.429936, Average norm of Jacobian: 4.309173, Norm of difference in interpretations: 1.427782
	Loss: 1.210263, Average norm of Jacobian: 8.409556, Norm of difference in interpretations: 2.448098
	Loss: 1.272747, Average norm of Jacobian: 9.952688, Norm of difference in interpretations: 2.591655
	Loss: 1.283171, Average norm of Jacobian: 18.107491, Norm of difference in interpretations: 3.731226
	Test set accuracy: (59.60%)
Epoch #3
	Loss: 1.354167, Average norm of Jacobian: 22.303371, N